<a href="https://colab.research.google.com/github/shashi3876/kaggle/blob/main/PodcastListenPredictCatboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

THis notebook contains the work done for the Kaggel playgorund competition named Predict Podcast Listening Time.

In [ ]:
# imports

import pandas as pd

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [ ]:
len(sample)

250000

In [ ]:
len(train)

750000

In [ ]:
test.columns

Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment'],
      dtype='object')

In [ ]:
train.columns

Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')

# EDA on the data

In [ ]:
train['Podcast_Name'].value_counts()

Podcast_Name
Tech Talks             22847
Sports Weekly          20053
Funny Folks            19635
Tech Trends            19549
Fitness First          19488
Business Insights      19480
Style Guide            19364
Game Day               19272
Melody Mix             18889
Criminal Minds         17735
Finance Focus          17628
Detective Diaries      17452
Crime Chronicles       17374
Athlete's Arena        17327
Fashion Forward        17280
Tune Time              17254
Business Briefs        17012
Lifestyle Lounge       16661
True Crime Stories     16373
Sports Central         16191
Digital Digest         16171
Humor Hub              16144
Mystery Matters        16002
Comedy Corner          15927
Joke Junction          15074
Wellness Wave          15009
Sport Spot             14778
Gadget Geek            14770
Home & Living          14686
Laugh Line             14673
Life Lessons           14464
World Watch            14043
Sound Waves            13928
Global News            13649
M

In [ ]:
test['Podcast_Name'].value_counts()

Podcast_Name
Tech Talks             7553
Sports Weekly          6633
Funny Folks            6621
Fitness First          6543
Style Guide            6469
Business Insights      6448
Tech Trends            6446
Melody Mix             6357
Game Day               6345
Criminal Minds         5980
Crime Chronicles       5867
Finance Focus          5859
Detective Diaries      5799
Fashion Forward        5787
Tune Time              5733
Lifestyle Lounge       5688
Business Briefs        5685
Athlete's Arena        5662
Humor Hub              5476
Sports Central         5424
True Crime Stories     5417
Mystery Matters        5268
Comedy Corner          5258
Digital Digest         5246
Joke Junction          5039
Wellness Wave          4948
Home & Living          4933
Laugh Line             4906
Gadget Geek            4849
Sport Spot             4844
Life Lessons           4823
Sound Waves            4689
Global News            4625
Mind & Body            4581
World Watch            4556
Current

In [ ]:
# Assuming train and test are pandas DataFrames
train_podcast_set = set(train['Podcast_Name'])
test_podcast_set = set(test['Podcast_Name'])

# Check if all Podcast_Name values in 'test' exist in 'train'
all_present = test_podcast_set.issubset(train_podcast_set)

if all_present:
    print("All Podcast_Name values from 'test' are present in 'train'.")
else:
    print("Not all Podcast_Name values from 'test' are present in 'train'.")
    # Find the missing podcast names
    missing_names = test_podcast_set - train_podcast_set
    print("Missing Podcast Names:", missing_names)

All Podcast_Name values from 'test' are present in 'train'.


In [ ]:
train['Publication_Day'].value_counts()

Publication_Day
Sunday       115946
Monday       111963
Friday       108237
Wednesday    107886
Thursday     104360
Saturday     103505
Tuesday       98103
Name: count, dtype: int64

In [ ]:
test['Number'] = pd.to_numeric(test['Episode_Title'].str.split().str[1])
train['Number'] = pd.to_numeric(train['Episode_Title'].str.split().str[1])

In [ ]:
test.sample(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Number
38881,788881,Music Matters,Episode 45,104.74,Music,90.29,Friday,Evening,71.10,2.0,Negative,45
240090,990090,Wellness Wave,Episode 91,8.78,Health,98.82,Friday,Evening,87.42,2.0,Neutral,91
72281,822281,Game Day,Episode 92,102.57,Sports,93.94,Wednesday,Afternoon,62.66,0.0,Negative,92
159151,909151,Music Matters,Episode 32,14.24,Music,84.13,Saturday,Morning,NaN,1.0,Negative,32
179005,929005,Crime Chronicles,Episode 48,74.96,True Crime,23.89,Thursday,Evening,57.00,1.0,Positive,48
198872,948872,Tune Time,Episode 43,77.45,Music,78.64,Saturday,Evening,NaN,2.0,Positive,43
58805,808805,Tech Talks,Episode 75,77.69,Technology,39.45,Saturday,Afternoon,NaN,3.0,Positive,75
58989,808989,Gadget Geek,Episode 25,46.36,Technology,75.71,Sunday,Night,56.08,3.0,Neutral,25
85703,835703,Business Briefs,Episode 58,60.93,Business,60.32,Friday,Night,NaN,1.0,Positive,58
119008,869008,True Crime Stories,Episode 27,NaN,True Crime,51.87,Friday,Evening,73.21,2.0,Negative,27


In [ ]:
train.sample(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Number
387390,387390,Business Insights,Episode 39,119.01,Business,38.55,Thursday,Night,68.31,2.0,Negative,63.78274,39
516937,516937,Business Briefs,Episode 73,44.39,Business,31.60,Sunday,Night,87.01,1.0,Neutral,37.93825,73
659285,659285,Sports Weekly,Episode 93,21.48,Sports,38.36,Friday,Afternoon,88.72,2.0,Neutral,8.54000,93
447830,447830,Sports Weekly,Episode 32,43.60,Sports,27.14,Wednesday,Evening,51.39,3.0,Negative,26.76000,32
571840,571840,Detective Diaries,Episode 69,31.97,True Crime,79.06,Friday,Afternoon,NaN,3.0,Neutral,9.86000,69
584088,584088,Sports Weekly,Episode 23,77.31,Sports,68.13,Wednesday,Morning,96.04,1.0,Neutral,57.05372,23
127255,127255,Detective Diaries,Episode 42,119.05,True Crime,87.34,Wednesday,Evening,NaN,1.0,Negative,95.23086,42
563779,563779,True Crime Stories,Episode 50,NaN,True Crime,29.67,Monday,Afternoon,NaN,0.0,Neutral,44.88606,50
639719,639719,Digital Digest,Episode 60,15.44,Technology,38.96,Tuesday,Evening,84.98,2.0,Positive,14.35553,60
18797,18797,Game Day,Episode 9,26.91,Sports,59.57,Monday,Afternoon,33.06,1.0,Positive,17.13247,9


In [ ]:
train.isna().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
Number                              0
dtype: int64

In [ ]:
test.isna().sum()

id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
Number                             0
dtype: int64

Checking whether the podcast)name and episode number combinations are unique.

In [ ]:
result_1 = train.groupby('Podcast_Name').agg({
    'Number': ['min', 'max']
})
result_2 = test.groupby('Podcast_Name').agg({
    'Number': ['min', 'max']
})

In [ ]:
merged=result_1.merge(result_2, on='Podcast_Name', suffixes=('_train', '_test'))
merged.head()

Number_train      Number_test     
                           min  max         min  max
Podcast_Name                                        
Athlete's Arena              1  100           1  100
Brain Boost                  1  100           1  100
Business Briefs              1  100           1  100
Business Insights            1  100           1  100
Comedy Corner                1  100           1  100

It seems like the same episode number may reappear for the same Podcast name over test and train.

## Add the percentage listened to each podcast

HEre, I'm adding a new feature which is the percentage listened to each podcast. The Listening_Time_minute/Episode_Length_minutes * 100 will be used as the output of the model. First I'll try whetehr this value is always less than 100%.

In [ ]:
train['Listened_Percentage']=train['Listening_Time_minutes']/train['Episode_Length_minutes']*100
train['Listened_Percentage'].describe()

count    6.629070e+05
mean              inf
std               NaN
min      0.000000e+00
25%      5.662961e+01
50%      6.983871e+01
75%      8.223749e+01
max               inf
Name: Listened_Percentage, dtype: float64

In [ ]:
# Get the top 20 highest values of 'Listened_Percentage'
top_20_percentage = train['Listened_Percentage'].nlargest(20)
print("Top 20 'Listened_Percentage' values:\n", top_20_percentage)

# Calculate the percentage of data with 'Listened_Percentage' above 100%
percentage_above_100 = (train[train['Listened_Percentage'] > 100].shape[0] / len(train)) * 100
print(f"\nPercentage of data with 'Listened_Percentage' > 100%: {percentage_above_100:.2f}%")


Top 20 'Listened_Percentage' values:
 150178            inf
250219    9417.741935
453791    7889.864865
642283    5454.347826
107984    2520.331984
486115    2514.474494
553429    2514.474494
674029    2514.474494
63965      943.644963
3859       937.400827
391491     918.609339
235546     891.434045
583239     878.768745
480516     854.113146
253190     842.317154
696739     836.517803
186316     813.714012
119498     779.497356
159896     779.497356
197372     779.497356
Name: Listened_Percentage, dtype: float64

Percentage of data with 'Listened_Percentage' > 100%: 0.34%


It seems that around 34% of the podcasts have a Listened_percentage greater than 100%. We'll drop these values from the training set. Then, we'll use the average Listened_percentage as the baseline for predictions based on the podcast name.

If the average Listened_percentage is null due to the unavailability of the episode length, we'll use the mean of the Listening_time_minutes for that podcast name as the prediction instead.

Since we're using a mean-based baseline, I'm not going to split the training set into training and validation sets for now.

In [ ]:
# Drop rows where Listened_percentage > 100%
train_cleaned = train[train['Listened_Percentage'] <= 100].copy()

# Group by Podcast_Name and compute:
# - average Listened_percentage
# - average Listening_time_minutes
grouped_stats = train_cleaned.groupby('Podcast_Name').agg({
    'Listened_Percentage': 'mean',
    'Listening_Time_minutes': 'mean'
}).reset_index()

# Optional: Rename columns for clarity
grouped_stats.rename(columns={
    'Listened_Percentage': 'Avg_Listened_Percentage',
    'Listening_Time_minutes': 'Avg_Listening_Time_minutes'
}, inplace=True)

# Preview
print(grouped_stats.head())
len(grouped_stats)

        Podcast_Name  Avg_Listened_Percentage  Avg_Listening_Time_minutes
0    Athlete's Arena                68.624694                   46.832812
1        Brain Boost                67.691371                   44.653527
2    Business Briefs                68.350878                   47.241432
3  Business Insights                67.257004                   44.161217
4      Comedy Corner                67.632612                   44.185392


48

In [ ]:
len(test.Podcast_Name.unique())

48

Now let's perform a lookup on the episode name to retrieve the Listening_Time_minutes column, and then generate the output results.

In [ ]:

test_merged = test.merge(grouped_stats, on='Podcast_Name', how='left')

def compute_listening_time(row):
    if pd.notnull(row['Episode_Length_minutes']):
        return row['Episode_Length_minutes'] * row['Avg_Listened_Percentage'] / 100
    else:
        return row['Avg_Listening_Time_minutes']

test_merged['Listening_Time_minutes'] = test_merged.apply(compute_listening_time, axis=1)

# Preview
print(test_merged[['Podcast_Name', 'Episode_Length_minutes', 'Avg_Listened_Percentage',
                   'Avg_Listening_Time_minutes', 'Listening_Time_minutes']].head())


          Podcast_Name  Episode_Length_minutes  Avg_Listened_Percentage  \
0  Educational Nuggets                   78.96                67.396264   
1          Sound Waves                   27.87                67.380592   
2        Joke Junction                   69.10                66.939189   
3        Comedy Corner                  115.39                67.632612   
4         Life Lessons                   72.32                67.568575   

   Avg_Listening_Time_minutes  Listening_Time_minutes  
0                   45.824576               53.216090  
1                   44.952358               18.778971  
2                   43.082075               46.254979  
3                   44.185392               78.041271  
4                   45.403412               48.865594  


In [ ]:
test_merged[['id','Listening_Time_minutes']].round(3).to_csv('submission.csv',index=False)

In [ ]:
train.columns

Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment', 'Listening_Time_minutes', 'Number',
       'Listened_Percentage'],
      dtype='object')

This resulted in a score of 13.97 which is an improvement form the previous value of 27.16 (Which was the score of the sample submission).

# Use of Catboost

Here, I will be using Catboost directly on the training data and obtain results.

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [ ]:
# Define columns to exclude from features
exclude_cols = ['id', 'Listened_Percentage', 'Number', 'Listening_Time_minutes']
target_col = 'Listening_Time_minutes'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id', 'Number'])

# Identify categorical features (assumes object types are categorical)
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Listening_Time_minutes'] = test_preds

# Optional: preview results
print(test[['Podcast_Name', 'Episode_Title', 'Listening_Time_minutes']].head())


0:	learn: 26.1531765	test: 26.1379438	best: 26.1379438 (0)	total: 184ms	remaining: 6m 8s
100:	learn: 13.1766039	test: 13.1373353	best: 13.1373353 (100)	total: 12.8s	remaining: 4m
200:	learn: 13.1100810	test: 13.0904830	best: 13.0904830 (200)	total: 24.8s	remaining: 3m 41s
300:	learn: 13.0696976	test: 13.0667004	best: 13.0667004 (300)	total: 36.7s	remaining: 3m 27s
400:	learn: 13.0318407	test: 13.0453842	best: 13.0453842 (400)	total: 48.6s	remaining: 3m 13s
500:	learn: 13.0010993	test: 13.0316119	best: 13.0316119 (500)	total: 1m	remaining: 3m 1s
600:	learn: 12.9725272	test: 13.0198654	best: 13.0198654 (600)	total: 1m 12s	remaining: 2m 49s
700:	learn: 12.9462424	test: 13.0115938	best: 13.0115938 (700)	total: 1m 25s	remaining: 2m 38s
800:	learn: 12.9226490	test: 13.0036676	best: 13.0036676 (800)	total: 1m 38s	remaining: 2m 26s
900:	learn: 12.9004710	test: 12.9979323	best: 12.9979323 (900)	total: 1m 50s	remaining: 2m 14s
1000:	learn: 12.8782376	test: 12.9902888	best: 12.9902888 (1000)	tota

# Fill the missing value before running the code

In [ ]:
train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
test['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)

train['Number_of_Ads'].fillna(train['Number_of_Ads'].median(), inplace=True)
train['Guest_Popularity_percentage'].fillna(train['Guest_Popularity_percentage'].median(), inplace=True)


test['Guest_Popularity_percentage'].fillna(test['Guest_Popularity_percentage'].median(), inplace=True)

/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_77909/1422714757.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_77909/1422714757.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermedi

In [ ]:
# Define columns to exclude from features
exclude_cols = ['id', 'Listened_Percentage', 'Number', 'Listening_Time_minutes']
target_col = 'Listening_Time_minutes'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id', 'Number'])

# Identify categorical features (assumes object types are categorical)
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Listening_Time_minutes'] = test_preds

# Optional: preview results
print(test[['Podcast_Name', 'Episode_Title', 'Listening_Time_minutes']].head())


0:	learn: 26.1434905	test: 26.1271362	best: 26.1271362 (0)	total: 174ms	remaining: 5m 48s


In [ ]:
test[['id','Listening_Time_minutes']].round(3).to_csv('submission_catboost_2.csv',index=False)

The above results yield a score of 13.134 on the submission.

#Hyperparameter Optimization

Here, I will apply hyperparameter optimization using Optuna to try and achieve better results from the CatBoost model.

In [ ]:
import optuna
from sklearn.metrics import mean_squared_error

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from math import sqrt

# Prepare training data
drop_cols = ['id', 'Listened_Percentage', 'Number', 'Listening_Time_minutes']
X = train.drop(columns=drop_cols)
y = train['Listening_Time_minutes']
cat_features = X.select_dtypes(include='object').columns.tolist()

# Split for evaluation
X_train,  y_train = X, y

# Define model
model = CatBoostRegressor(verbose=100, cat_features=cat_features)

# Define parameter grid (keep it small to start with)
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3,7],
    'iterations': [1000,2000]
}
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'bagging_temperature': [0, 0.5, 1],
    'random_strength': [1, 5, 10],
    'bootstrap_type': ['Bayesian', 'Bernoulli'],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}
'''
param_grid = {
    'depth': [6, 8],
    'learning_rate': [0.03, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000]
}
'''
# Grid search
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error',
                           cv=3,
                           n_jobs=2)

grid_search.fit(X_train, y_train)

# Best results
print("✅ Best RMSE:", -grid_search.best_score_)
print("🎯 Best Parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_


In [ ]:
test

In [ ]:
# Prepare the test data (drop same columns as in training)
X_test = test.drop(columns=['id', 'Number'])

# Predict using the best model from GridSearchCV
test_preds = best_model.predict(X_test)

# Add predictions to the test DataFrame
test['Listening_Time_minutes'] = test_preds

test[['id','Listening_Time_minutes']].round(3).to_csv('submission_catboost_5.csv',index=False)


# Code to save the inetrmediate results

In [ ]:
import itertools
import os
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load / prepare training data
drop_cols = ['id', 'Listened_Percentage', 'Number', 'Listening_Time_minutes']
X = train.drop(columns=drop_cols)
y = train['Listening_Time_minutes']
cat_features = X.select_dtypes(include='object').columns.tolist()

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Parameter grid
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'random_strength': [1, 5, 10],
    'bootstrap_type': ['Bayesian', 'Bernoulli'],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}

# Convert grid to list of dicts (one per combination)
def expand_grid(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combos = list(itertools.product(*values))
    return [dict(zip(keys, combo)) for combo in combos]

all_combinations = expand_grid(param_grid)

# File to cache results
results_file = "catboost_grid_results_new.csv"
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
else:
    results_df = pd.DataFrame(columns=list(param_grid.keys()) + ["rmse"])

# Run only for new combinations
new_results = []

for params in all_combinations:
    # Check if this combination has already been run
    mask = (results_df[list(params)] == pd.Series(params)).all(axis=1)
    if mask.any():
        print(f"✅ Skipping already-run: {params}")
        continue

    print(f"🚀 Running: {params}")
    try:
        model = CatBoostRegressor(cat_features=cat_features, verbose=0, **params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, preds))
        print(f"→ RMSE: {rmse:.4f}")

        # Save result after each run
        result_row = {**params, 'rmse': rmse}
        results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)
        results_df.to_csv(results_file, index=False)
        print(f"💾 Result saved to {results_file}")

    except Exception as e:
        print(f"❌ Error with params {params}: {e}")

# Append new results and save
if new_results:
    results_df = pd.concat([results_df, pd.DataFrame(new_results)], ignore_index=True)
    results_df.to_csv(results_file, index=False)
    print(f"💾 Results saved to {results_file}")

# Show best
best = results_df.sort_values('rmse').iloc[0]
print("\n🏆 Best Config:")
print(best)


🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSE: 13.0514
💾 Result saved to catboost_grid_results_new.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}


/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_45190/1429438751.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)


→ RMSE: 13.0442
💾 Result saved to catboost_grid_results_new.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree'}
→ RMSE: 13.0505
💾 Result saved to catboost_grid_results_new.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise'}
→ RMSE: 13.0398
💾 Result saved to catboost_grid_results_new.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSE: 13.0659
💾 Result saved to catboost_grid_results_new.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}
→ RMSE: 13.0572
💾 Result saved to catboost_grid_results_new.csv
🚀 Running:

# Run with best model

In [ ]:
# Define columns to exclude from features
exclude_cols = ['id', 'Listened_Percentage', 'Number', 'Listening_Time_minutes']
target_col = 'Listening_Time_minutes'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id', 'Number'])

# Identify categorical features (assumes object types are categorical)
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=10,
    random_strength=1,
    l2_leaf_reg=3,
    bootstrap_type='Bernoulli',
    grow_policy='Depthwise',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Listening_Time_minutes'] = test_preds

# Optional: preview results
print(test[['Podcast_Name', 'Episode_Title', 'Listening_Time_minutes']].head())

0:	learn: 26.5315405	test: 26.5157774	best: 26.5157774 (0)	total: 154ms	remaining: 5m 6s
100:	learn: 13.2037454	test: 13.1902423	best: 13.1902423 (100)	total: 11.1s	remaining: 3m 28s
200:	learn: 13.0843855	test: 13.0977030	best: 13.0977030 (200)	total: 20.8s	remaining: 3m 6s
300:	learn: 13.0472225	test: 13.0813059	best: 13.0813059 (300)	total: 32.9s	remaining: 3m 5s
400:	learn: 13.0167374	test: 13.0691575	best: 13.0691575 (400)	total: 41.3s	remaining: 2m 44s
500:	learn: 12.9070726	test: 13.0398989	best: 13.0398989 (500)	total: 50s	remaining: 2m 29s
600:	learn: 12.7261526	test: 13.0060931	best: 13.0060931 (600)	total: 59.9s	remaining: 2m 19s
700:	learn: 12.5670939	test: 12.9817659	best: 12.9817659 (700)	total: 1m 9s	remaining: 2m 9s
800:	learn: 12.4245381	test: 12.9644726	best: 12.9644726 (800)	total: 1m 20s	remaining: 2m
900:	learn: 12.2906919	test: 12.9498514	best: 12.9498514 (900)	total: 1m 30s	remaining: 1m 50s
1000:	learn: 12.1659436	test: 12.9375451	best: 12.9375451 (1000)	total: 

In [ ]:
test[['id','Listening_Time_minutes']].round(3).to_csv('submission_catboost_6.csv',index=False)

# Using XGB models